### les importations neccessaires :

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

### les deux méthodes suivantes peuvent importer les image dans jupyter notebook




In [ ]:
#importation de dataset 

dataset = keras.preprocessing.image_dataset_from_directory(
  '/home/amin/Documents/test', batch_size=64, image_size=(200, 200))

In [ ]:
#importation de dataset en batches avec application de preprocess 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory='/home/amin/Documents/test', target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

### cette fonction affiche 10 images (1 batch)







In [ ]:


imgs,labels=next(train_batches)


def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    
plotImages(imgs)
print(labels)    
    


### on peut ajouter des couches dans le modèle afin de faire quelques opération de preprocessing telque :
- redimensionement 
- scaling
- CenterCrop

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

result = resize_and_rescale(image)
_ = plt.imshow(result)

### on peut aussi ajouter des couches dans le modèle afin de faire quelques opération de preprocessing telque :
- RandomFlip :  retourner aléatoirement chaque image horizontalement et verticalement
- RandomRotation :  faire tourner chaque image de façon aléatoire.
- RandomContrast :  ajuster le contraste d'une ou plusieurs images par un facteur aléatoire
- RandomCrop :  recadrage aléatoire des images à la hauteur et à la largeur cibles
- RandomZoom :  Zoomer aléatoirement chaque image pendant la formation 

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

# Add the image to a batch
image = tf.expand_dims(image, 0)

plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

### l'augmentation de données peut etre une partie de modele 
Lors de l'exportation de modèle . les couches de prétraitement seront enregistrées avec le reste de votre modèle. Si on déploy , normalisera automatiquement les images

In [ ]:
model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
    # le reste de modele
    
])



